In [1]:
# Here are a set of libraries we imported to complete this assignment. 
# Feel free to use these or equivalent libraries for your implementation
# If you can run this cell without any errors, you're ready to go.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # this is used for the plot the graph 

import seaborn as sns # used for plot interactive graph.
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC  
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import sklearn.metrics.cluster as smc
from matplotlib import pyplot

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_curve, auc

from sklearn.naive_bayes import GaussianNB


# scikit-learn k-fold cross-validation
from numpy import array
from sklearn.model_selection import KFold


import os
import itertools
import random 

%matplotlib inline

random.seed(148) 

from tsfresh import extract_features

In [2]:
# make it dark mode (delete it later)
# !jt -l
# jt -l

In [3]:
#!jt -t <theme name>
#!jt -t chesterish
#jt -t chesterish

In [4]:
#Enable the toolbar: !jt -t [theme name] -T

In [5]:
# Helper function allowing you to export a graph
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [6]:
# Helper function that allows you to draw nicely formatted confusion matrices
def draw_confusion_matrix(y, yhat, classes):
    '''
        Draws a confusion matrix for the given target and predictions
        Adapted from scikit-learn example.
    '''
    plt.cla()
    plt.clf()
    matrix = confusion_matrix(y, yhat)
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    num_classes = len(classes)
    plt.xticks(np.arange(num_classes), classes, rotation=90)
    plt.yticks(np.arange(num_classes), classes)
    
    fmt = 'd'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matrix[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

1. Merge Datasets and Effectively Link information - Useful information for this project will come from disparate datasets. You will need to effectively merge them into a single dataframe for analysis

Can combine:
Brand Average Retail Price
Brand total sales
Brand total units

Combine on: Brand and Month to get the columns:
Brand, Month:
    ARP, vs Prior Period (Average Retail Price), Total Sales (total sales), Total Units, vs Prior Period (total units)

In [7]:
#data_1 = pd.read_csv('data\BrandAverageRetailPrice.csv')
#data_2 = pd.read_csv('data\BrandDetails.csv')
#data_3 = pd.read_csv('data\BrandTotalSales.csv')
#data_4 = pd.read_csv('data\BrandTotalUnits.csv')

#combine based on brand
#https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html
#df3 = pd.merge(df1, df2)
#combine_1 = pd.merge(data_1, data_2)
#combine_2 = pd.merge(data_3, data_4)
#data = pd.merge(combine_1, combine_2)

#to check all of data
#data.to_csv('total_combine_data.csv', index=False)

In [8]:
#the , in the thousands space makes the columns an object
#data_1
brand_average_retail_price = pd.read_csv('data\BrandAverageRetailPrice.csv', thousands=",")
#data_2
brand_details = pd.read_csv('data\BrandDetails.csv')
#data_3
brand_total_sales = pd.read_csv('data\BrandTotalSales.csv', thousands=",")
#data_4
brand_total_units = pd.read_csv('data\BrandTotalUnits.csv', thousands=",")
#Need to rename prior period as it is not being added onto, they are being combined (in 1 and 4)
#data_4
brand_total_units = brand_total_units.rename(columns={"vs. Prior Period": "vs Prior Period unit"})

#combine based on brand
#https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html
#df3 = pd.merge(df1, df2)
#Need outer join to keep every month
#Difference between price and unit # in prior period
combine_1 = pd.merge(brand_average_retail_price, brand_total_units, how='left', suffixes=('_apr', '_units'))
#combine_2 = pd.merge(data_3, data_4)
data_per_month = pd.merge(combine_1, brand_total_sales, left_on = ['Brands', 'Months'], right_on = ['Brand', 'Months'], how= 'left', \
                          suffixes=(False, '_total_sales'))

#brand got copied over onto another column, need to delete that
del data_per_month['Brand']


#So first issue is the data in its current form isn't really useful to us, 
#so let's do some conversion of our data.
#First convert our months to datetime
#data_per_month['Months'] = pd.to_datetime(data_per_month['Months'])
# #Total units is too large currently to convert to a float
# #need to trim it first then convert to float
# data_per_month['Total Units'] = data_per_month['Total Units'].str[8:]
# data_per_month['Total Units'] = pd.to_numeric(data_per_month['Total Units'])


data_per_month.info()

#to check all of data
data_per_month.to_csv('data_per_month.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27211 entries, 0 to 27210
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Brands                27211 non-null  object 
 1   Months                27211 non-null  object 
 2   ARP                   25279 non-null  float64
 3   vs. Prior Period      24499 non-null  float64
 4   Total Units           25279 non-null  float64
 5   vs Prior Period unit  24893 non-null  float64
 6   Total Sales ($)       25279 non-null  float64
dtypes: float64(5), object(2)
memory usage: 1.7+ MB


Need to handle BrandDetails differents as it only tracks their items and their price and then details the Total Sales, Total Units, ARP

Note for later: flavor, is flavored == Null, ask later

In [9]:
#Do later as it does not fit at the moment

In [10]:
#Head Method
data_per_month.head()
#arp is average retail price

,Brands,Months,ARP,vs. Prior Period,Total Units,vs Prior Period unit,Total Sales ($)
0,#BlackSeries,08/2020,15.684913,NaN,1616.339004,NaN,25352.135918
1,#BlackSeries,09/2020,NaN,-1.000000,NaN,-1.000000,NaN
2,#BlackSeries,01/2021,13.611428,NaN,715.532838,NaN,9739.423400
3,#BlackSeries,02/2021,11.873182,-0.127705,766.669135,0.071466,9102.802187
4,#BlackSeries,03/2021,NaN,-1.000000,NaN,-1.000000,NaN


In [11]:
#Describe Method
data_per_month.describe()

,ARP,vs. Prior Period,Total Units,vs Prior Period unit,Total Sales ($)
count,25279.000000,24499.000000,2.527900e+04,24893.000000,2.527900e+04
mean,22.679732,-0.065028,2.886210e+04,0.267441,4.093729e+05
std,19.802724,0.388923,1.617156e+05,3.293738,1.596024e+06
min,0.000000,-1.000000,3.842953e+00,-1.000000,0.000000e+00
25%,10.512827,-0.088073,7.169135e+02,-0.349745,1.390320e+04
50%,17.033051,-0.011649,3.605059e+03,-0.054400,6.210080e+04
75%,31.505612,0.045232,1.564044e+04,0.241073,2.473270e+05
max,700.874984,12.645741,5.248082e+06,250.792020,4.036351e+07


In [12]:
#Info method
data_per_month.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27211 entries, 0 to 27210
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Brands                27211 non-null  object 
 1   Months                27211 non-null  object 
 2   ARP                   25279 non-null  float64
 3   vs. Prior Period      24499 non-null  float64
 4   Total Units           25279 non-null  float64
 5   vs Prior Period unit  24893 non-null  float64
 6   Total Sales ($)       25279 non-null  float64
dtypes: float64(5), object(2)
memory usage: 1.7+ MB


2. Develop basic Time Series Feature Extraction Plan - develop a series of standard timeseries features to augment your dataset and enable timeseries predictive models.

In [13]:
#Need to split the data before we add on the features:
#Everything but the last month of a brand = train
#Last month of each brand = test

#only input the averages into the training set not the test set
#as to not leak values from the test to train and get overfitting

#https://stackoverflow.com/questions/36923494/pandas-dataframe-use-previous-row-value-for-complicated-if-conditions-to-deter

test_data = []
train_data = []

d = {'Brands': [], 'Months': [], 'ARP': [], 'vs. Prior Period': [], 'Total Units': [], 'vs Prior Period unit': [], 'Total Sales ($)': []}
test_data = pd.DataFrame(data=d)
train_data = pd.DataFrame(data=d)

print(test_data)
print(train_data)
print(data_per_month)

for index, row in data_per_month.iterrows():
    #row = dict containing all data for that row, not row number
    #that would be index
    next_row = index + 1
    #need .iloc
    if (next_row >= len(data_per_month)):
        #Need to check to see if any of the items are null, if they are, add in the averges
        
        test_data = test_data.append(data_per_month.iloc[index])
    elif (data_per_month.iloc[next_row]['Brands'] != data_per_month.iloc[index]['Brands']):
        test_data = test_data.append(data_per_month.iloc[index])
    else:
        train_data = train_data.append(data_per_month.iloc[index])

#print(train_data)
#print(test_data)
#to check all of data
train_data.to_csv('train_data_per_month.csv', index=False)
test_data.to_csv('test_data_per_month.csv', index=False)

Empty DataFrame
Columns: [Brands, Months, ARP, vs. Prior Period, Total Units, vs Prior Period unit, Total Sales ($)]
Index: []
Empty DataFrame
Columns: [Brands, Months, ARP, vs. Prior Period, Total Units, vs Prior Period unit, Total Sales ($)]
Index: []
              Brands   Months        ARP  vs. Prior Period  Total Units  \
0       #BlackSeries  08/2020  15.684913               NaN  1616.339004   
1       #BlackSeries  09/2020        NaN         -1.000000          NaN   
2       #BlackSeries  01/2021  13.611428               NaN   715.532838   
3       #BlackSeries  02/2021  11.873182         -0.127705   766.669135   
4       #BlackSeries  03/2021        NaN         -1.000000          NaN   
...              ...      ...        ...               ...          ...   
27206  Zuma Topicals  08/2019  31.598214               NaN   312.515336   
27207  Zuma Topicals  09/2019  37.860964          0.198199   464.306316   
27208  Zuma Topicals  10/2019  34.546154         -0.087552   348.057905

In [14]:
#Now, need to add in missing values to test data
#Columns:
#0: Brand
#1: Months
#2: ARP
#3: vs. Prior Period
#4: Total Units
#5: vs Prior Period unit
#6: Total Sales ($)

#Need the () for copy!!
filled_in_test_data = test_data.copy().reset_index()

# d = {'Brands': [], 'Months': [], 'ARP': [], 'vs. Prior Period': [], 'Total Units': [], 'vs Prior Period unit': [], 'Total Sales ($)': []}
# filled_in_test_data = pd.DataFrame(data=d)

#project 1

#Taking the average as it feels better

new_ARP = data_per_month["ARP"].mean()
new_vs_Prior_Period = data_per_month["vs. Prior Period"].mean()
new_total_units = data_per_month["Total Units"].mean()
new_vs_unit = data_per_month["vs Prior Period unit"].mean()
new_total_sales = data_per_month["Total Sales ($)"].mean()

filled_in_test_data["ARP"].fillna(new_ARP, inplace=True)
#to replace the -1's
#Don't replace the -1's!!!
#filled_in_test_data["ARP"].replace({-1: new_ARP}, inplace=True)

filled_in_test_data["vs. Prior Period"].fillna(new_vs_Prior_Period, inplace=True)
#filled_in_test_data["vs. Prior Period"].replace({-1: new_vs_Prior_Period}, inplace=True)

filled_in_test_data["Total Units"].fillna(new_total_units, inplace=True)
#filled_in_test_data["Total Units"].replace({-1: new_total_units}, inplace=True)

filled_in_test_data["vs Prior Period unit"].fillna(new_vs_unit, inplace=True)
#filled_in_test_data["vs Prior Period unit"].replace({-1: new_vs_unit}, inplace=True)

filled_in_test_data["Total Sales ($)"].fillna(new_total_sales, inplace=True)
#filled_in_test_data["Total Sales ($)"].replace({-1: new_total_sales}, inplace=True)

        
filled_in_test_data.to_csv('filled_in_test_data.csv', index=False)

In [15]:
#now add in missing values to Train data, but leave out the test data
#Columns:
#0: Brand
#1: Months
#2: ARP
#3: vs. Prior Period
#4: Total Units
#5: vs Prior Period unit
#6: Total Sales ($)

#Need the () for copy!!
filled_in_train_data = train_data.copy().reset_index()

# d = {'Brands': [], 'Months': [], 'ARP': [], 'vs. Prior Period': [], 'Total Units': [], 'vs Prior Period unit': [], 'Total Sales ($)': []}
# filled_in_test_data = pd.DataFrame(data=d)

#project 1

new_ARP = test_data["ARP"].mean()
new_vs_Prior_Period = test_data["vs. Prior Period"].mean()
new_total_units = test_data["Total Units"].mean()
new_vs_unit = test_data["vs Prior Period unit"].mean()
new_total_sales = test_data["Total Sales ($)"].mean()

filled_in_train_data["ARP"].fillna(new_ARP, inplace=True)
#to replace the -1's
#don't replace the -1's!!
#filled_in_train_data["ARP"].replace({-1: new_ARP}, inplace=True)

filled_in_train_data["vs. Prior Period"].fillna(new_vs_Prior_Period, inplace=True)
#filled_in_train_data["vs. Prior Period"].replace({-1: new_vs_Prior_Period}, inplace=True)

filled_in_train_data["Total Units"].fillna(new_total_units, inplace=True)
#filled_in_train_data["Total Units"].replace({-1: new_total_units}, inplace=True)

filled_in_train_data["vs Prior Period unit"].fillna(new_vs_unit, inplace=True)
#filled_in_train_data["vs Prior Period unit"].replace({-1: new_vs_unit}, inplace=True)

filled_in_train_data["Total Sales ($)"].fillna(new_total_sales, inplace=True)
#filled_in_train_data["Total Sales ($)"].replace({-1: new_total_sales}, inplace=True)

        
filled_in_train_data.to_csv('filled_in_train_data.csv', index=False)

Last months, 
Last quarter's, 
Last year's average

In [16]:
# Austyn
# #Last months
# #Last quarter's 
# #Last year's average

# #Last months:
# months = ['Jan', 'Feb', 'March', 'Apr', "May", 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
# months_num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# #augmenting based on vs Prior Period
# #For total sales: divd based on Total Units or ARP, figure out later
# #For each row, 
# #if the row above does not have a month prior, add in a month 
# #add in a check like a column that sayss added in and mark it yes

# #New dataset
additional_month_train = []

d = {'Brands': [], 'Months': [], 'ARP': [], 'vs. Prior Period': [], 'Total Units': [], 'vs Prior Period unit': [], 'Total Sales ($)': []}
additional_month_train = pd.DataFrame(data=d)

train_filled_in_copy = filled_in_train_data.copy().reset_index()

for row, index in train_filled_in_copy.iterrows():
    #index = dict containing all data for that row, 
    #not row number that would be row
#     print("row is: ")
#     print(row)
#     print("index is")
#     print(index)
#     print("index [1] is")
#     print(index[1])
#     print("index[1][0] is")
#     print(index[0][1])
#     print("index[0][1] is")
#     print(index[1][0])
#     print("index[Month] is:")
#     print(index["Months"])
#     #Need string for columns, not indiviual cells
#     print("index[Months].split('/') is")
#     print(index["Months"].split('/'))
#     broken_apart_month = index["Months"].split('/')
#     print(broken_apart_month[0])
#     print(broken_apart_month[1])
    
    
    prev_row = row - 1
    #need .iloc
    if (prev_row < 0):
        # print("Previous row is 0")
        # Need to check to see if any of the items are null, if they are, add in the averges
        # 0th row (the month we are adding before the first row in the dataset)
        new_brand = train_filled_in_copy.iloc[0]['Brands']
        broken_apart_month = index["Months"].split('/')
        previous_month = str(int(broken_apart_month[0])- 1)
        # print("before_previous_month")
        # print(previous_month)
        year = broken_apart_month[1]
        #print(year)
        previous_month_int = int(broken_apart_month[0])- 1
        if (previous_month_int == 0):
            previous_month = str(12)
            year = str(int(broken_apart_month[1]) - 1)
        combined_previous_month = previous_month + "/" + year
        # print(combined_previous_month)
        
#         #Testing for 2nd if
#         #Working good
#         print("Month 1 line:")
#         month_num = int(train_filled_in_copy.iloc[0]['Months'].split('/')[0])
#         print(month_num)
#         print(type(month_num))
#         print("Month string")
#         month_string = str(train_filled_in_copy.iloc[0]['Months'].split('/')[0])
#         print(month_string)
#         print(type(month_string))
        
#         if(int(train_filled_in_copy.iloc[0]['Months'].split('/')[0]) == 
#            int(train_filled_in_copy.iloc[0]['Months'].split('/')[0])):
#             print("True, test passed")
#         else:
#             print("Test Failed")
        
        
        original_arp = float(index["ARP"])
        #print("Original arp is")
        #print(original_arp)
        #new_vs_Prior_Period = data_per_month["vs. Prior Period"].mean()
        new_vs_prior_period = data_per_month["vs. Prior Period"].mean()
        #print("old prior period is")
        #print(index["vs. Prior Period"])
        how_much_new = float(original_arp) * float(index["vs. Prior Period"])
        #print("how much new")
        #print(how_much_new)
        before_arp_int = float(original_arp) + float(how_much_new)
        new_arp = before_arp_int
        #print("new_arp")
        #print(new_arp)
        
        
        original_total_units = float(index["Total Units"])
        new_prior_units = data_per_month["vs Prior Period unit"].mean()
        how_much_new = float(original_total_units) * float(index["vs Prior Period unit"])
        new_total_units_int = float(original_total_units) + float(how_much_new)
        new_total_units = new_total_units_int
        
        
        new_total_sales = new_arp * new_total_units
        
        #new_vs_unit = data_per_month["vs Prior Period unit"].mean()
#         a_row = pd.Series([new_brand,
#                            combined_previous_month,
#                            new_arp, 
#                            new_vs_prior_period, 
#                            new_total_units,
#                            new_prior_units,
#                            new_total_sales])
        a_dict = {'Brands': new_brand, 
                  'Months': combined_previous_month, 
                  'ARP': new_arp, 
                  'vs. Prior Period': new_vs_prior_period, 
                  'Total Units': new_total_units, 
                  'vs Prior Period unit': new_prior_units, 
                  'Total Sales ($)': new_total_sales
                 }
        row_df = pd.DataFrame([a_dict])
        additional_month_train = additional_month_train.append(row_df, ignore_index=True)
        #supposed to be:
        #Brand: #Black Series
        #Months: 7/2020
        #ARP: 6.75
        #vs. Prior Period: 
        #Total Units: 934
        #vs Prior Period unit:
        #Total Sales ($): 6304.5
#         print("New row values are:")
#         print("Brand is:")
#         print(new_brand)
#         print("New month is")
#         print(combined_previous_month)
#         print("ARP is")
#         print(new_arp)
#         print("new prior period is")
#         print(new_vs_prior_period)
#         print("new total units is")
#         print(new_total_units)
#         print("New prior units are:")
#         print(new_prior_units)
#         print("new total sales are")
#         print(new_total_sales)
        #1st row
        
        original_brand = index["Brands"]
        original_months = index["Months"]
        original_arp = index["ARP"]
        original_vs_prior = index["vs. Prior Period"]
        original_total_unit = index["Total Units"]
        original_prior_units = index["vs Prior Period unit"]
        original_sales = index["Total Sales ($)"]
#         b_row = pd.Series([original_brand,
#                            original_months,
#                            original_arp, 
#                            original_vs_prior, 
#                            original_total_unit,
#                            original_prior_units,
#                            original_sales])
        b_dict = {'Brands': original_brand, 
                  'Months': original_months, 
                  'ARP': original_arp, 
                  'vs. Prior Period': original_vs_prior, 
                  'Total Units': original_total_unit, 
                  'vs Prior Period unit': original_prior_units, 
                  'Total Sales ($)': original_sales}
        b_row_df = pd.DataFrame([b_dict])
        additional_month_train = additional_month_train.append(b_row_df, ignore_index=True)
#         print("Original row is:")
#         print(train_filled_in_copy.iloc[row])
        
        #print("---------------------------------------------------------------------------")
        
        
    #If the previous entry does not have the same brand or the months are not 1 apart
    #data_per_month.iloc[next_row]['Brands'] != data_per_month.iloc[index]['Brands']
#     (str(additional_month_train.iloc[-1]['Brands']) !=
#           str(train_filled_in_copy.iloc[row]['Brands'])) or 
    elif ((str(additional_month_train.iloc[-1]['Brands']) ==
          str(train_filled_in_copy.iloc[row]['Brands'])) and 
          (int(additional_month_train.iloc[-1]['Months'].split('/')[0]) != 
           int(train_filled_in_copy.iloc[row]['Months'].split('/')[0]) - 1)):
        
        if(int(train_filled_in_copy.iloc[row]['Months'].split('/')[0]) - 1 == 0 and
           int(additional_month_train.iloc[-1]['Months'].split('/')[0]) == 12):
            continue
        
        new_brand = filled_in_train_data.iloc[row]['Brands']
        broken_apart_month = index["Months"].split('/')
        previous_month = str(int(broken_apart_month[0])- 1)
        previous_month_int = int(broken_apart_month[0])- 1
        year = broken_apart_month[1]
        if (previous_month_int == 0):
            previous_month = str(12)
            year = str(int(broken_apart_month[1]) - 1)
        combined_previous_month = previous_month + "/" + year
        #print(combined_previous_month)
        original_arp = float(index["ARP"])
        new_vs_prior_period = data_per_month["vs. Prior Period"].mean()
        how_much_new = float(original_arp) * float(index["vs. Prior Period"])
        before_arp_int = float(original_arp) + float(how_much_new)
        new_arp = before_arp_int
        original_total_units = float(index["Total Units"])
        new_prior_units = data_per_month["vs Prior Period unit"].mean()
        how_much_new = float(original_total_units) * float(index["vs Prior Period unit"])
        new_total_units_int = float(original_total_units) + float(how_much_new)
        new_total_units = new_total_units_int
        new_total_sales = new_arp * new_total_units
        #new_vs_unit = data_per_month["vs Prior Period unit"].mean()
#         a_row = pd.Series([new_brand,
#                            combined_previous_month,
#                            new_arp, 
#                            new_vs_prior_period, 
#                            new_total_units,
#                            new_prior_units,
#                            new_total_sales])
#         row_df = pd.DataFrame([a_row])
        a_dict = {'Brands': new_brand, 
                  'Months': combined_previous_month, 
                  'ARP': new_arp, 
                  'vs. Prior Period': new_vs_prior_period, 
                  'Total Units': new_total_units, 
                  'vs Prior Period unit': new_prior_units, 
                  'Total Sales ($)': new_total_sales}
        row_df = pd.DataFrame([a_dict])
        additional_month_train = additional_month_train.append(row_df, ignore_index=True)
        
        original_brand = index["Brands"]
        original_months = index["Months"]
        original_arp = index["ARP"]
        original_vs_prior = index["vs. Prior Period"]
        original_total_unit = index["Total Units"]
        original_prior_units = index["vs Prior Period unit"]
        original_sales = index["Total Sales ($)"]
        
#         b_row = pd.Series([original_brand,
#                            original_months,
#                            original_arp, 
#                            original_vs_prior, 
#                            original_total_unit,
#                            original_prior_units,
#                            original_sales])
#         b_row_df = pd.DataFrame([b_row])
        b_dict = {'Brands': original_brand, 
                  'Months': original_months, 
                  'ARP': original_arp, 
                  'vs. Prior Period': original_vs_prior, 
                  'Total Units': original_total_unit, 
                  'vs Prior Period unit': original_prior_units, 
                  'Total Sales ($)': original_sales}
        b_row_df = pd.DataFrame([b_dict])
        additional_month_train = additional_month_train.append(b_row_df, ignore_index=True)
        
#         print("New row values are:")
#         print("Brand is:")
#         print(new_brand)
#         print("New month is")
#         print(combined_previous_month)
#         print("ARP is")
#         print(new_arp)
#         print("new prior period is")
#         print(new_vs_prior_period)
#         print("new total units is")
#         print(new_total_units)
#         print("New prior units are:")
#         print(new_prior_units)
#         print("new total sales are")
#         print(new_total_sales)
        
#         print("Original row is:")
#         print(additional_month_train)
        
#         print("---------------------------------------------------------------------------")
        
        
#     #If the previous entry is not the same brand
#     elif(data_per_month.iloc[next_row]['Brands'] != train_data.iloc[index]['Brands']):
#         new_brand = filled_in_train_data.iloc[index]['Brands']
#         broken_apart_month = index["Months"].split('/')
#         previous_month = str(int(broken_apart_month[0])- 1)
#         year = broken_apart_month[1]
#         if (previous_month == 0):
#             previous_month = str(12)
#             year = str(int(broken_apart_month[1]) - 1)
#         combined_previous_month = previous_month + "/" + year
#         original_arp = float(index["ARP"])
#         new_vs_prior_period = data_per_month["vs. Prior Period"].mean()
#         how_much_new = float(original_arp) * float(index["vs. Prior Period"])
#         before_arp_int = float(original_arp) + float(how_much_new)
#         new_arp = before_arp_int
#         original_total_units = float(index["Total Units"])
#         new_prior_units = data_per_month["vs Prior Period unit"].mean()
#         how_much_new = float(original_total_units) * float(index["vs Prior Period unit"])
#         new_total_units_int = float(original_total_units) + float(how_much_new)
#         new_total_units = new_total_units_int
#         new_total_sales = new_arp * new_total_units
#         #new_vs_unit = data_per_month["vs Prior Period unit"].mean()
#         additional_month_train = additional_month_train.append([new_brand,
#                                                                combined_previous_month,
#                                                                new_arp, 
#                                                                new_vs_prior_period, 
#                                                                new_total_units,
#                                                                new_prior_units,
#                                                                new_total_sales])
#         additional_month_train = additional_month_train.append(train_data.iloc[index])
    
    #need case of first entry of the brand to add in
    elif(str(additional_month_train.iloc[-1]['Brands']) !=
          str(train_filled_in_copy.iloc[row]['Brands'])):
        new_brand = filled_in_train_data.iloc[row]['Brands']
        broken_apart_month = index["Months"].split('/')
        previous_month = str(int(broken_apart_month[0])- 1)
        year = broken_apart_month[1]
        previous_month_int = int(broken_apart_month[0])- 1
        if (previous_month_int == 0):
            previous_month = str(12)
            year = str(int(broken_apart_month[1]) - 1)
        combined_previous_month = previous_month + "/" + year
        original_arp = float(index["ARP"])
        new_vs_prior_period = data_per_month["vs. Prior Period"].mean()
        how_much_new = float(original_arp) * float(index["vs. Prior Period"])
        before_arp_int = float(original_arp) + float(how_much_new)
        new_arp = before_arp_int
        original_total_units = float(index["Total Units"])
        new_prior_units = data_per_month["vs Prior Period unit"].mean()
        how_much_new = float(original_total_units) * float(index["vs Prior Period unit"])
        new_total_units_int = float(original_total_units) + float(how_much_new)
        new_total_units = new_total_units_int
        new_total_sales = new_arp * new_total_units
        a_dict = {'Brands': new_brand, 
                  'Months': combined_previous_month, 
                  'ARP': new_arp, 
                  'vs. Prior Period': new_vs_prior_period, 
                  'Total Units': new_total_units, 
                  'vs Prior Period unit': new_prior_units, 
                  'Total Sales ($)': new_total_sales
                 }
        row_df = pd.DataFrame([a_dict])
        additional_month_train = additional_month_train.append(row_df, ignore_index=True)
        original_brand = index["Brands"]
        original_months = index["Months"]
        original_arp = index["ARP"]
        original_vs_prior = index["vs. Prior Period"]
        original_total_unit = index["Total Units"]
        original_prior_units = index["vs Prior Period unit"]
        original_sales = index["Total Sales ($)"]
        b_dict = {'Brands': original_brand, 
                  'Months': original_months, 
                  'ARP': original_arp, 
                  'vs. Prior Period': original_vs_prior, 
                  'Total Units': original_total_unit, 
                  'vs Prior Period unit': original_prior_units, 
                  'Total Sales ($)': original_sales}
        b_row_df = pd.DataFrame([b_dict])
        additional_month_train = additional_month_train.append(b_row_df, ignore_index=True)
        
        
    #Easy case, when the previous row has the same brand, and the month is one back
    else:
        original_brand = index["Brands"]
        original_months = index["Months"]
        original_arp = index["ARP"]
        original_vs_prior = index["vs. Prior Period"]
        original_total_unit = index["Total Units"]
        original_prior_units = index["vs Prior Period unit"]
        original_sales = index["Total Sales ($)"]
        
#         b_row = pd.Series([original_brand,
#                            original_months,
#                            original_arp, 
#                            original_vs_prior, 
#                            original_total_unit,
#                            original_prior_units,
#                            original_sales])
#         b_row_df = pd.DataFrame([b_row])
        b_dict = {'Brands': original_brand, 
                  'Months': original_months, 
                  'ARP': original_arp, 
                  'vs. Prior Period': original_vs_prior, 
                  'Total Units': original_total_unit, 
                  'vs Prior Period unit': original_prior_units, 
                  'Total Sales ($)': original_sales}
        b_row_df = pd.DataFrame([b_dict])
        additional_month_train = additional_month_train.append(b_row_df, ignore_index=True)
        
#         print("Original row is:")
#         print(additional_month_train)
#         print("---------------------------------------------------------------------------")


#to check all of data
additional_month_train.to_csv('additional_month_train.csv', index=False)

In [17]:
#train_copy = train_data.copy().reset_index()
# train_copy.dropna(subset=['Months'], inplace=True)
data_per_month['Months'] = pd.to_datetime(data_per_month['Months'])

In [18]:
# https://www.geeksforgeeks.org/split-a-text-column-into-two-columns-in-pandas-dataframe/
# train_data.Months.str.split('/')[1]
# train_copy['month'], train_copy['year'] = pd.DataFrame(train_data.Months.str.split('/',1).tolist(), columns = ['month', 'year'])
# train_copy
#train_2 = pd.DataFrame(train_data.Months.str.split('/', 1).tolist(), columns = ['month', 'year'])
additional_month_train['month'] = pd.DatetimeIndex(additional_month_train['Months']).month
additional_month_train['year'] = pd.DatetimeIndex(additional_month_train['Months']).year
additional_month_train

,Brands,Months,ARP,vs. Prior Period,Total Units,vs Prior Period unit,Total Sales ($),month,year
0,#BlackSeries,7/2020,7.440011,-0.065028,934.666384,0.267441,6953.928122,7,2020
1,#BlackSeries,08/2020,15.684913,-0.525658,1616.339004,-0.421739,25352.135918,8,2020
2,#BlackSeries,09/2020,19.621934,-1.000000,28690.238607,-1.000000,378071.185836,9,2020
3,#BlackSeries,12/2020,6.456470,-0.065028,413.764989,0.267441,2671.461312,12,2020
4,#BlackSeries,01/2021,13.611428,-0.525658,715.532838,-0.421739,9739.423400,1,2021
...,...,...,...,...,...,...,...,...,...
27624,Zuma Topicals,7/2019,14.988356,-0.065028,180.715542,0.267441,2708.628955,7,2019
27625,Zuma Topicals,08/2019,31.598214,-0.525658,312.515336,-0.421739,9874.926545,8,2019
27626,Zuma Topicals,09/2019,37.860964,0.198199,464.306316,0.485707,17579.084649,9,2019
27627,Zuma Topicals,10/2019,34.546154,-0.087552,348.057905,-0.250370,12024.061922,10,2019


In [19]:
annual_avg_add = additional_month_train.copy()

annual_avg_add.dropna(subset=['ARP'], inplace=True)
if 'index' in annual_avg_add.columns:
    annual_avg_add.drop(['index'], axis=1, inplace=True)

# yearly average: count brands and year, then add sum of ARP, then divide
grouped = annual_avg_add.groupby(['Brands', 'year'])
annual_avg_add['annual_avg'] = grouped['ARP'].transform('sum') / grouped['Brands'].transform('count')
annual_avg_add.to_csv('annual_avg_add.csv', index=False)
annual_avg_add

,Brands,Months,ARP,vs. Prior Period,Total Units,vs Prior Period unit,Total Sales ($),month,year,annual_avg
0,#BlackSeries,7/2020,7.440011,-0.065028,934.666384,0.267441,6953.928122,7,2020,12.300832
1,#BlackSeries,08/2020,15.684913,-0.525658,1616.339004,-0.421739,25352.135918,8,2020,12.300832
2,#BlackSeries,09/2020,19.621934,-1.000000,28690.238607,-1.000000,378071.185836,9,2020,12.300832
3,#BlackSeries,12/2020,6.456470,-0.065028,413.764989,0.267441,2671.461312,12,2020,12.300832
4,#BlackSeries,01/2021,13.611428,-0.525658,715.532838,-0.421739,9739.423400,1,2021,12.742305
...,...,...,...,...,...,...,...,...,...,...
27624,Zuma Topicals,7/2019,14.988356,-0.065028,180.715542,0.267441,2708.628955,7,2019,31.168738
27625,Zuma Topicals,08/2019,31.598214,-0.525658,312.515336,-0.421739,9874.926545,8,2019,31.168738
27626,Zuma Topicals,09/2019,37.860964,0.198199,464.306316,0.485707,17579.084649,9,2019,31.168738
27627,Zuma Topicals,10/2019,34.546154,-0.087552,348.057905,-0.250370,12024.061922,10,2019,31.168738


In [20]:
#yearly average for each brand
#split
#yearly average
#Total sales for each brand

#1 row of data for each data point, each row has all info
#Data, all necessary info in one row
# multiple data training point for 1 branch, mulitple data branch
#For each brand

#overall data also: seasonalities

In [21]:
# #rolling_df = additional_month_train.copy()
# # Define a custom function which groups by time (using the index)
# rolling_average_percent_change = additional_month_train.copy()
# rolling_df = additional_month_train.copy()

# rolling_average_percent_change = rolling_df.assign(moving_score=rolling_df.groupby(['Brands'], as_index=False)[['Total Sales ($)']]
#                            .rolling(720, min_periods=0).mean().fillna(0)
#                            .reset_index(0, drop=True))
# #additional_month_train.head(50)

# rolling_average_percent_change["prcnt chng"] = rolling_average_percent_change["Total Sales ($)"].pct_change(1)
# rolling_average_percent_change.to_csv('rolling_average_percent_change.csv', index=False)

In [22]:
#Add in a rolling average

In [39]:
#rolling_df = additional_month_train.copy()
# Define a custom function which groups by time (using the index)
rolling_average_percent_change = annual_avg_add.copy()
#rolling_df = annual_avg_add.copy()

#rolling_average_percent_change = rolling_average_percent_change.assign(rolling_average_units=rolling_average_percent_change.groupby(['Brands'], as_index=False)[['Total Units']].rolling(720, min_periods=0).mean().fillna(0).reset_index(0, drop=True))
rolling_average_percent_change = rolling_average_percent_change.assign(rolling_average_units=rolling_average_percent_change.groupby(['Brands'], as_index=False)[['Total Units']]
                                                                       .rolling(720, min_periods=0)
                                                                       .mean()
                                                                       #.fillna(0)
                                                                       .reset_index(0, drop=True)
                                                                      )



rolling_average_percent_change["percent change units"] = rolling_average_percent_change["Total Units"].pct_change(1)

rolling_df = annual_avg_add.copy()

rolling_average_percent_change = rolling_average_percent_change.assign(rolling_average_sales=rolling_average_percent_change.groupby(['Brands'], as_index=False)[['Total Sales ($)']].rolling(720, min_periods=0).mean().fillna(0).reset_index(0, drop=True))
#additional_month_train.head(50)

rolling_average_percent_change["percent change sales"] = rolling_average_percent_change["Total Sales ($)"].pct_change(1)

rolling_average_percent_change.to_csv('rolling_average_percent_change.csv', index=False)

rolling_average_percent_change

,Brands,Months,ARP,vs. Prior Period,Total Units,vs Prior Period unit,Total Sales ($),month,year,annual_avg,rolling_average_units,percent change units,rolling_average_sales,percent change sales
0,#BlackSeries,7/2020,7.440011,-0.065028,934.666384,0.267441,6953.928122,7,2020,12.300832,934.666384,NaN,6953.928122,NaN
1,#BlackSeries,08/2020,15.684913,-0.525658,1616.339004,-0.421739,25352.135918,8,2020,12.300832,1275.502694,0.729322,16153.032020,2.645729
2,#BlackSeries,09/2020,19.621934,-1.000000,28690.238607,-1.000000,378071.185836,9,2020,12.300832,10413.747998,16.750137,136792.416625,13.912794
3,#BlackSeries,12/2020,6.456470,-0.065028,413.764989,0.267441,2671.461312,12,2020,12.300832,7913.752246,-0.985578,103262.177797,-0.992934
4,#BlackSeries,01/2021,13.611428,-0.525658,715.532838,-0.421739,9739.423400,1,2021,12.742305,6474.108364,0.729322,84557.626918,2.645729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27624,Zuma Topicals,7/2019,14.988356,-0.065028,180.715542,0.267441,2708.628955,7,2019,31.168738,180.715542,-0.700920,2708.628955,-0.513824
27625,Zuma Topicals,08/2019,31.598214,-0.525658,312.515336,-0.421739,9874.926545,8,2019,31.168738,246.615439,0.729322,6291.777750,2.645729
27626,Zuma Topicals,09/2019,37.860964,0.198199,464.306316,0.485707,17579.084649,9,2019,31.168738,319.179065,0.485707,10054.213383,0.780174
27627,Zuma Topicals,10/2019,34.546154,-0.087552,348.057905,-0.250370,12024.061922,10,2019,31.168738,326.398775,-0.250370,10546.675518,-0.316002


In [40]:
# df = rolling_average_percent_change
# # Replace the column names as appropriate
# # Leave the column names empty
# # if you don't have them in your data
# features = extract_features(df, column_id="Brands", column_sort="Months")
# features

In [ ]:
# features["ARP__median"]

## Brand Features Engineering

I have a theory that it's important to determine if a company offers inhaleable and edible products as part of their product inventory so I'm going to create binary categorical features.

In [25]:
value = 0
value1 = 0

brand_details_copy = brand_details.copy()

if 'Inhaleables' in brand_details_copy['Category L1'].values:
    value = 1
if 'Edibles' in brand_details_copy['Category L1'].values:
    value1 = 1
 
rolling_average_percent_change['Inhaleables'] = value
rolling_average_percent_change['Edible'] = value1


rolling_average_percent_change

,Brands,Months,ARP,vs. Prior Period,Total Units,vs Prior Period unit,Total Sales ($),month,year,rolling_average_sales,percent change sales,Inhaleables,Edible
0,#BlackSeries,7/2020,7.440011,-0.065028,934.666384,0.267441,6953.928122,7,2020,6953.928122,NaN,1,0
1,#BlackSeries,08/2020,15.684913,-0.525658,1616.339004,-0.421739,25352.135918,8,2020,16153.032020,2.645729,1,0
2,#BlackSeries,09/2020,19.621934,-1.000000,28690.238607,-1.000000,378071.185836,9,2020,136792.416625,13.912794,1,0
3,#BlackSeries,12/2020,6.456470,-0.065028,413.764989,0.267441,2671.461312,12,2020,103262.177797,-0.992934,1,0
4,#BlackSeries,01/2021,13.611428,-0.525658,715.532838,-0.421739,9739.423400,1,2021,84557.626918,2.645729,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27624,Zuma Topicals,7/2019,14.988356,-0.065028,180.715542,0.267441,2708.628955,7,2019,2708.628955,-0.513824,1,0
27625,Zuma Topicals,08/2019,31.598214,-0.525658,312.515336,-0.421739,9874.926545,8,2019,6291.777750,2.645729,1,0
27626,Zuma Topicals,09/2019,37.860964,0.198199,464.306316,0.485707,17579.084649,9,2019,10054.213383,0.780174,1,0
27627,Zuma Topicals,10/2019,34.546154,-0.087552,348.057905,-0.250370,12024.061922,10,2019,10546.675518,-0.316002,1,0


I also believe that a total count of the number of products the brand offers is also a useful feature to include. Fortunately that's easy enough to determine!

In [26]:
productcount = (brand_details.Brand == '101 Cannabis Co.').count()

productcount

144977

In [27]:
rolling_average_percent_change['ProdCount'] = productcount

rolling_average_percent_change.head()

,Brands,Months,ARP,vs. Prior Period,Total Units,vs Prior Period unit,Total Sales ($),month,year,rolling_average_sales,percent change sales,Inhaleables,Edible,ProdCount
0,#BlackSeries,7/2020,7.440011,-0.065028,934.666384,0.267441,6953.928122,7,2020,6953.928122,NaN,1,0,144977
1,#BlackSeries,08/2020,15.684913,-0.525658,1616.339004,-0.421739,25352.135918,8,2020,16153.032020,2.645729,1,0,144977
2,#BlackSeries,09/2020,19.621934,-1.000000,28690.238607,-1.000000,378071.185836,9,2020,136792.416625,13.912794,1,0,144977
3,#BlackSeries,12/2020,6.456470,-0.065028,413.764989,0.267441,2671.461312,12,2020,103262.177797,-0.992934,1,0,144977
4,#BlackSeries,01/2021,13.611428,-0.525658,715.532838,-0.421739,9739.423400,1,2021,84557.626918,2.645729,1,0,144977


The result is starting to look like a pretty darn good dataframe! We now have merged and engineered timeseries features, along with brand-level features included in our dataframe. 

To complete this work the next steps will be to: 

1. finalize our feature selection plan
2. consolidate these steps into a concise for loop for all brands and then append them into a single dataframe
3. finalize an imputation strategy
4. You can then treat the dataset like a typical regression problem where 'TotalSales' or 'TotalUnits' can be the label you predict on
5. As always report your metrics! (and speaking of metrics, I found this handy-dandy helper functin that spits out a bunch of useful ones for you...)

3. Run some basic statistics on your variables including correlations with labels and report findings - Particularly once you employ PCA and other ‘black box’ methods, the descriptive power of any of your features will effectively disappear. Still you want to report out meaningful correlations toCookies to help them flag key indicators they can employ (this step will also be helpful for you in flagging potential co-linearities).

In [ ]:
#correlation matrix: enough
corr_matrix = rolling_average_percent_change.corr()
#visualize data if there is some nonlinear relationships between features

corr_matrix

In [ ]:
sns.heatmap(corr_matrix)

4. Create additional data feature extraction plan and implement a comprehensive pipeline to execute it - Determine and execute a plan to process your data for modeling and then implement a pipeline to execute it. Specifically:
    1. Determine which fields to retain and which to drop.
    2. For those you retain, determine a categorization strategy
    3. Determine an imputation strategy (you should choose more than one imputation method depending on the specifics of your data
    4. Augment at least one feature, ideally a feature cross, or non-linear transition e.
        Determine a strategy for scaling features

5. Implement a single pipeline to execute this transformation

6. Document your data strategy in your report. Provide an explanation or justification for why you chose the data you did, and also detail any experiments you ran and the results

7. Implement a basic Linear Regression predictive model - With your newly pipelined data find and interpret important features (e.g. using regression and associated p-values). If there are any collinearities be careful when incorporating them into the regression.

8. Implement Principle Component Analysis (PCA) - Since your resulting dataframe is likely to be high-dimensionality, employ PCA to reduce the complexity of your dataframe

9. Employ an ensemble method to your predictive model exercise - Leverage an ensemble learning method to generate an optimized prediction model

10. Cross-Validate your training results - Employ K-Fold Cross-validation to your training regimen for both ensemble and single regression models. (Optional: employ a stratifiedshufflesplit as well to ensure equitable distribution along a key parameter)

11. Employ a GridSearch method to optimize your parameters - Leverage gridsearch or an equivalent parameter tuning approach to optimize parameters to your predictive model (Note: you can likely merge the gridsearch and cross-validation steps)

12. Experiment with your own custom models and report out your highest performing model - For this part of the project you have free range to employ any of the tools you’ve learned in class, along with any additional tools or techniques you research independently to see how you can do.